In [ ]:
import boto
import dask.dataframe as dd
import pandas as pd
import os
import re
import logging
from dask.delayed import delayed
from distributed import Client
from functools import partial

In [ ]:
s3 = boto.connect_s3()
bucket = s3.get_bucket('data.mindhypertrophy.com')

In [ ]:
client = Client('localhost:8786')

In [ ]:
# match all the loan tapes from the year 2011
pattern = re.compile('\d{8}/loan_stats.csv')
host = f's3://{bucket.name}'
urls = [key.generate_url(expires_in=600) for key in bucket.list() if pattern.search(key.name)]
print(f'found {len(urls)} urls')

In [ ]:
def read_csv_async(url):
    print(f'reading {url}')
    df = pd.read_csv(url, index_col=False)
    return df

In [ ]:
# create a list of 'future' data frame objects
future_frames = [delayed(read_csv_async)(url) for url in urls]
# create a 'future' DataFrame object
df_future = dd.from_delayed(future_frames)

In [ ]:
%%time
df = df_future.compute()

In [ ]:
df.head()